In [4]:
import csv
import json
from sage.all import Partitions, Partition, binomial
import sage.libs.lrcalc.lrcalc as lrcalc
from tqdm import tqdm
import numpy as np

# PARAMETERS: set dimensions for the box.
k = 5   # maximum part size (number of columns)
n = 5   # maximum number of parts (number of rows)

###############################################################################
# Function: Generate all partitions fitting in a k x n box.
###############################################################################
def partitions_in_box(k, n):
    """
    Returns a list of Partition objects corresponding to partitions that fit 
    in an n x k box (i.e. at most n parts and largest part <= k).
    Each partition is padded with zeros to length n.
    """
    parts = []
    # Maximum possible weight is k * n.
    for weight in range(k * n + 1):
        for p in Partitions(weight):
            if len(p) <= n and (len(p) == 0 or p[0] <= k):
                # Pad with zeros to length n.
                p_list = list(p) + [0] * (n - len(p))
                parts.append(Partition(p_list))
    return parts

###############################################################################
# Function: Convert a partition into an n x k binary matrix.
###############################################################################
def partition_to_matrix(p, n, k):
    """
    Given a Partition object p, first pad it to length n (if needed), then 
    return an n x k matrix (as a list of lists) where each row i has 1's in the 
    first p[i] entries and 0's elsewhere.
    """
    p_list = list(p)
    if len(p_list) < n:
        p_list = p_list + [0] * (n - len(p_list))
    matrix = []
    for i in range(n):
        row = [1 if j < p_list[i] else 0 for j in range(k)]
        matrix.append(row)
    return matrix

###############################################################################
# Function: Convert a matrix (list of lists) to a string.
###############################################################################
def matrix_to_str(mat):
    """
    Convert a matrix (list of lists) to a string.
    Each row is space-separated; rows are separated by semicolons.
    """
    return ";".join(" ".join(map(str, row)) for row in mat)

###############################################################################
# Function: Convert a partition to lattice path
###############################################################################
def lattice_path(p, n, k):
    """
    Compute the lattice path representation for a partition p in a k x n box.
    
    The lattice path is constructed as follows:
      - Convert the partition p to a list (if it isn't already) and pad it to length n.
      - For each row i (0-indexed), compute the position as:
            pos = (k - p_list[i]) + (i+1)
        (interpreting positions as 1-indexed).
      - Create a binary string of length n+k where a '1' is placed in each of these positions,
        and '0' elsewhere.
    """
    # Convert to list and pad to length n.
    p_list = list(p)
    if len(p_list) < n:
        p_list = p_list + [0] * (n - len(p_list))
    
    # Compute the 1-indexed positions for '1's.
    ones_positions = [(k - p_list[i]) + (i + 1) for i in range(n)]
    
    # Initialize binary word with '0's of length n+k.
    binary_word = ['0'] * (n + k)
    
    # Set positions to '1' (converting to 0-indexed).
    for pos in ones_positions:
        if 1 <= pos <= (n + k):
            binary_word[pos - 1] = '1'
    
    return "".join(binary_word)


###############################################################################
# MAIN DATA GENERATION
###############################################################################
all_parts = partitions_in_box(k, n)
N = len(all_parts)
total_triples = N ** 3
print("Total partitions in box:", N)
print("Total triples (before filtering):", total_triples)

# Prepare lists for CSV and JSON records.
csv_rows = []
json_records = []

# Write CSV header.
csv_header = ["lambda_matrix", "mu_matrix", "nu_matrix", "lr_coefficient"]

# Use tqdm to show progress.
pbar = tqdm(total=total_triples, desc="Processing triples")

# Triple nested loop over all partitions.
for lam in all_parts:
    for mu in all_parts:
        for nu in all_parts:
            pbar.update(int(1))
            # Only consider valid triples: |lambda| = |mu| + |nu|
            if sum(mu) + sum(nu) == sum(lam):
                # Compute LR coefficient.
                lr = lrcalc.lrcoef(list(lam), list(mu), list(nu))
                # Convert partitions to their matrix representations.
                lam_mat = partition_to_matrix(lam, n, k)
                mu_mat = partition_to_matrix(mu, n, k)
                nu_mat = partition_to_matrix(nu, n, k)
                # Convert matrices to string.
                lam_str = matrix_to_str(lam_mat)
                mu_str = matrix_to_str(mu_mat)
                nu_str = matrix_to_str(nu_mat)
                # Convert partitions to lattice paths
                lam_path = lattice_path(lam, n,k)
                mu_path = lattice_path(mu, n,k)
                nu_path = lattice_path(nu, n,k)
                # Prepare the row/record.
                row = [lam_str, mu_str, nu_str, lam_path, mu_path, nu_path, int(lr)]
                csv_rows.append(row)
                json_records.append({
                    "lambda_matrix": lam_str,
                    "mu_matrix": mu_str,
                    "nu_matrix": nu_str,
                    "lambda_path": lam_path,
                    "mu_path": mu_path,
                    "nu_path": nu_path,
                    "lr_coefficient": int(lr)
                })

pbar.close()

# Export CSV.
csv_filename = "lr_coefficients_matrix.csv"
with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_header)
    for row in csv_rows:
        writer.writerow(row)
print("CSV data written to", csv_filename)

# Export JSON.
json_filename = "lr_coefficients_matrix.json"
with open(json_filename, "w") as jsonfile:
    json.dump(json_records, jsonfile, indent=int(2))
print("JSON data written to", json_filename)


Total partitions in box: 252
Total triples (before filtering): 16003008


Processing triples: 100%|██████████| 16003008/16003008 [01:05<00:00, 242901.92it/s]


CSV data written to lr_coefficients_matrix.csv
JSON data written to lr_coefficients_matrix.json


In [ ]:
import json
from sage.all import Partitions, Partition, Integer
import sage.libs.lrcalc.lrcalc as lrcalc
from tqdm.notebook import tqdm


# PARAMETERS: set dimensions for the box.
k = 4   # maximum part size (number of columns)
n = 4   # maximum number of parts (number of rows)

###############################################################################
# Helper: Recursively convert Sage Integer objects to native Python ints.
###############################################################################
def convert_sage_ints(obj):
    if isinstance(obj, Integer):
        return int(obj)
    elif isinstance(obj, list):
        return [convert_sage_ints(x) for x in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_sage_ints(x) for x in obj)
    elif isinstance(obj, dict):
        return {convert_sage_ints(k): convert_sage_ints(v) for k, v in obj.items()}
    else:
        return obj

###############################################################################
# Function: Generate all partitions that fit in a k x n box.
###############################################################################
def partitions_in_box(k, n):
    parts = []
    # The maximum possible weight is k * n.
    for weight in range(int(k * n) + 1):
        for p in Partitions(weight):
            if len(p) <= n and (len(p) == 0 or p[0] <= k):
                # Pad the partition with zeros to length n.
                p_list = list(p) + [int(0)] * (n - len(p))
                parts.append(Partition(p_list))
    return parts

###############################################################################
# Function: Convert a partition into a binary matrix.
###############################################################################
def partition_to_matrix(p, rows, cols):
    p_list = list(p)
    if len(p_list) < rows:
        p_list = p_list + [int(0)] * (rows - len(p_list))
    matrix = []
    for i in range(rows):
        row = [1 if j < p_list[i] else 0 for j in range(cols)]
        matrix.append(row)
    return matrix

###############################################################################
# Function: Convert a matrix (list of lists) to a string.
###############################################################################
def matrix_to_str(mat):
    return ";".join(" ".join(map(str, row)) for row in mat)

###############################################################################
# Function: Compute Durfee square size from a padded partition list.
###############################################################################
def durfee_square_size(p_list):
    d = 0
    for i, part in enumerate(p_list, start=1):
        if int(part) >= int(i):
            d = i
        else:
            break
    return int(d)

###############################################################################
# Function: Compute features for a partition.
###############################################################################
def compute_partition_features(p, n, k):
    # Pad the partition to length n.
    p_list = list(p)
    if len(p_list) < n:
        p_list = p_list + [int(0)] * (n - len(p_list))
    features = {}
    # "list": stored as a JSON array of native ints.
    features["list"] = [int(x) for x in p_list]
    # "length": number of nonzero parts.
    features["length"] = int(len([x for x in p_list if x != 0]))
    # "area": sum of parts.
    features["area"] = int(sum(p_list))
    # "durfee": computed from the padded list.
    features["durfee"] = durfee_square_size(p_list)
    # "lattice_path": for each row i, position = (k - p_list[i]) + (i+1) (1-indexed)
    ones_positions = [(int(k) - int(p_list[i])) + (i + 1) for i in range(n)]
    binary_word = ['0'] * (n + k)
    for pos in ones_positions:
        if 1 <= pos <= (n + k):
            binary_word[int(pos) - 1] = '1'
    features["lattice_path"] = "".join(binary_word)
    # "matrix" representation (n x k)
    mat = partition_to_matrix(p, n, k)
    features["matrix"] = mat
    features["matrix_str"] = matrix_to_str(mat)
    # Conjugate: compute the conjugate partition and pad its list to length k.
    p_conj = Partition(list(p).copy()).conjugate()
    p_conj_list = list(p_conj)
    if len(p_conj_list) < k:
        p_conj_list = p_conj_list + [int(0)] * (k - len(p_conj_list))
    features["conjugate_list"] = [int(x) for x in p_conj_list]
    # Conjugate matrix: dimensions k x n.
    conj_mat = partition_to_matrix(p_conj, k, n)
    features["conjugate_matrix"] = conj_mat
    features["conjugate_matrix_str"] = matrix_to_str(conj_mat)
    # Frobenius coordinates: use p.frobenius() which returns (arms, legs)
    try:
        frob = p.frobenius()
        features["frobenius"] = {"arms": [int(x) for x in frob[0]], "legs": [int(x) for x in frob[1]]}
    except Exception as e:
        features["frobenius"] = None
    return features

###############################################################################
# MAIN: Write features and LR coefficients directly to JSON file.
###############################################################################
all_parts = partitions_in_box(k, n)
N = len(all_parts)
print("Total partitions in box:", N)
# ... [previous code remains unchanged] ...

total_triples = int(N)**3
print("Total triples (before filtering):", total_triples)

json_filename = "lr_coefficients_full_features_k{}_n{}.json".format(k, n)
with open(json_filename, "w") as f:
    f.write("[\n")
    first_record = True
    # Use tqdm with miniters to update less frequently.
    pbar = tqdm(total=total_triples, desc="Processing triples", miniters=1000, mininterval=1)
    for lam in all_parts:
        lam_features = compute_partition_features(lam, n, k)
        for mu in all_parts:
            mu_features = compute_partition_features(mu, n, k)
            for nu in all_parts:
                pbar.update(int(1))
                if int(sum(mu)) + int(sum(nu)) == int(sum(lam)):
                    nu_features = compute_partition_features(nu, n, k)
                    lr = lrcalc.lrcoef(list(lam), list(mu), list(nu))
                    lr_int = int(lr)
                else:
                    nu_features = compute_partition_features(nu, n, k)
                    lr_int = 0
                record = {
                    "lambda": lam_features,
                    "mu": mu_features,
                    "nu": nu_features,
                    "lr_coefficient": lr_int
                }
                record = convert_sage_ints(record)
                if not first_record:
                    f.write(",\n")
                else:
                    first_record = False
                f.write(json.dumps(record, indent=int(2)))
    pbar.close()
    f.write("\n]\n")
print("JSON file written to", json_filename)



Total partitions in box: 252
Total triples (before filtering): 16003008



Processing triples:  62%|██████▏   | 9928862/16003008 [32:38<18:17, 5536.98it/s]